In [1]:
# Path setup to resolve package imports (dynamic project root)
import sys, os

# Derive project root from current notebook directory: <project>/notebook
notebook_dir = os.getcwd()
project_root = os.path.dirname(notebook_dir)

if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [ ]:
from src.agents.tools.db_tools import db_tool_manager
# Test tool manager
tools = db_tool_manager.get_tools()
print(f"✅ {len(tools)} tools ready")
print("Tool names:", [t.name for t in tools])


✅ 4 tools ready
Tool names: ['list_all_tables', 'get_table_schema', 'preview_sql', 'execute_sql']


In [ ]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from src.config.prompt import system_prompt

checkpointer = MemorySaver()

gptLLM = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",temperature=0
)

agent = create_agent(
    model=gptLLM,
    tools = tools,  # Add your tools here when ready
    system_prompt=system_prompt,
    checkpointer=checkpointer  # Use your custom ReAct prompt
)

config = {"configurable": {"thread_id": "user_123"}}


In [11]:
question = "Employees in Sales department HumanResources"
result = agent.invoke({
    "messages": [{"role": "user", "content": question}]
}, config=config)

print(f"\nAgent: {result['messages'][-1].content}")

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised NotFound: 404 models/gemini-1.5-flash is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 4.0 seconds as it raised NotFound: 404 models/gemini-1.5-flash is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 8.0 seconds as it raised NotFound: 404 models/gemini-1.5-flash is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods..
Retrying langchain_google_genai.chat_models._chat_with_retry.

KeyboardInterrupt: 